In [5]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences

C:\anaconda\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
import jieba
import re
import os

In [7]:
data = pd.read_csv("sentiment_analysis_trainingset.csv", encoding="utf-8") 

In [8]:
text = data['content']

In [9]:
def seg_words_write(contents):
    contents_segs = []
    stopwords = getStopwords()
    f = open('contents.txt','wb')
    for content in contents:
        c = re.sub(' ', '', content)
        segs = list(jieba.lcut(c))
        sentence_segment = []
        for word in segs:
            if word not in stopwords:
                sentence_segment.append(word)
        contents_segs.append(" ".join(sentence_segment))
        f.write(" ".join(sentence_segment).encode('utf-8'))
    f.close
    return contents_segs

In [10]:
def seg_words(contents):
    contents_segs = []
    stopwords = getStopwords()
    for content in contents:
        c = re.sub(' ', '', content)
        segs = list(jieba.lcut(c))
        sentence_segment = []
        for word in segs:
            if word not in stopwords:
                sentence_segment.append(word)
        contents_segs.append(" ".join(sentence_segment))
    return contents_segs

In [11]:
def getStopwords():
    stopwords = []
    with open("chineseStopWords.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            stopwords.append(line.strip())
    stopwords.append('"')
    stopwords.append('\n')
    stopwords.append('～')
    return stopwords

In [12]:
def sentence_vectorize(size,contents,model):
    contents_vector = np.zeros((len(contents),size))
    for i in range(len(contents)):
        segs = contents[i].split()
        vector_sum = np.zeros(size)
        for j in range(len(segs)):
            try:
                vector_sum = vector_sum + model[segs[j]]
            except KeyError:
                pass
        contents_vector[i] = vector_sum
    return contents_vector

In [13]:
def word2vec(contents, seg_contents_exist = False, size = 256, min_count=5, iter=10):
    if(seg_contents_exist == False):
        if os.path.exists("content.txt"):
            os.remove("content.txt")
        else:
            pass
        contents_segs = seg_words_write(contents)
    else:
        contents_segs = seg_words(contents)
    model = Word2Vec(PathLineSentences("contents.txt"),size=size, window=10, min_count=min_count, iter=iter)
    X = sentence_vectorize(size,contents_segs,model)
    return X

In [ ]:
contents_segs = seg_words(text)

In [ ]:
model = Word2Vec(PathLineSentences("contents.txt"),size=1000, window=10, min_count=5, iter=10)

In [12]:
X = sentence_vectorize(1000,contents_segs,model)

In [33]:
Word2Vec()

In [51]:
class Word_to_Vec():
    def __init__(self,embedder = Word2Vec(),size = 0,contents_segs = []):
        self.embedder = embedder
        self.size = size
    
    def fit(self, X, seg_contents_exist = False, size = 256, min_count=5, iter=10):
        self.size = 256
        if(seg_contents_exist == False):
            if os.path.exists("content.txt"):
                os.remove("content.txt")
            else:
                pass
            contents_segs = seg_words_write(X)
        else:
            contents_segs = seg_words(X)
        self.embedder = Word2Vec(PathLineSentences("contents.txt"),size=size, window=10, min_count=min_count, iter=iter)
    
    def transform(self, X):
        contents_segs = seg_words(X)
        return sentence_vectorize(self.size,contents_segs,self.embedder)
    
    def fit_transform(self, X, seg_contents_exist = False, size = 256, min_count=5, iter=10):
        size = 256
        if(seg_contents_exist == False):
            if os.path.exists("content.txt"):
                os.remove("content.txt")
            else:
                pass
            contents_segs = seg_words_write(X)
        else:
            contents_segs = seg_words(X)
        self.embedder = Word2Vec(PathLineSentences("contents.txt"),size=size, window=10, min_count=min_count, iter=iter)
        return sentence_vectorize(self.size,contents_segs,self.embedder)